# Quailty of BinaryInteraction Datasets

In this process, Quality of both BinaryInteraction datasets will be checked and if it is necessary it will improved

Null values and duplicates will be checked

User and Movie size will be checked

The representativeness of the training dataset will be checked and improved.

Organized data will be save as pkl file for future use


In [1]:
#Importing libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
#Printing library versions
print('numpy Version: ' + np.__version__)
print('pandas Version: ' + pd.__version__)

numpy Version: 1.16.5
pandas Version: 0.25.1


In [3]:
#Reading raw rating data from pkl file
ratingDf = pd.read_pickle("../Data/pkl/1M/RawData/Rating.pkl")
ratingDf

,UserId,MovieId,Rating
0,0,0,3.5
1,0,1,3.5
2,0,2,1.5
3,0,3,4.5
4,0,4,4.5
...,...,...,...
1000888,10072,12665,3.0
1000889,10072,6417,3.0
1000890,10072,9689,3.0
1000891,10072,22032,3.0


In [4]:
#Checking is there any nan value in ratingDf
ratingDf.isnull().values.any()

False

In [5]:
#Check if any duplicate user-movie pairs exist in datframe
ratingDf[['UserId', 'MovieId']].duplicated().any()

False

In [6]:
#Calculating number of users in raw data
userSize = ratingDf['UserId'].unique().shape[0]
userSize

10073

In [7]:
#Calculating number of movies in raw data
movieSize = ratingDf['MovieId'].unique().shape[0]
movieSize

22033

In [8]:
#Reading raw movie data from pkl file
movie = pd.read_pickle("../Data/pkl/1M/RawData/Movie.pkl")
movie

,MovieId,Title
0,0,Three Colors: Blue (Trois couleurs: Bleu) (1993)
1,1,Kalifornia (1993)
2,2,Weekend at Bernie's (1989)
3,3,Better Off Dead... (1985)
4,4,Waiting for Guffman (1996)
...,...,...
22028,22028,London Paris New York (2012)
22029,22029,Wild Zero (2000)
22030,22030,Mr. Accident (2000)
22031,22031,Max Steel (2016)


In [9]:
#Checking if movieSize is true
movie.shape[0]

22033

In [10]:
#Reading BinaryInteraction InteractedOnly training data from pkl file
trainingDf = pd.read_pickle("../Data/pkl/1M/BinaryInteraction/InteractedOnly/Training.pkl")
trainingDf

,UserId,MovieId,Interaction
0,9608,768,1
1,7750,2229,1
2,6702,3129,1
3,9108,208,1
4,9684,7155,1
...,...,...,...
750664,1722,1830,1
750665,4394,6984,1
750666,981,3083,1
750667,4624,958,1


In [11]:
#Reading BinaryInteraction InteractedOnly validation data from pkl file
validationDf = pd.read_pickle("../Data/pkl/1M/BinaryInteraction/InteractedOnly/Validation.pkl")
validationDf

,UserId,MovieId,Interaction
0,718,1903,1
1,2830,3528,1
2,9138,179,1
3,4026,504,1
4,8566,7612,1
...,...,...,...
125107,4291,251,1
125108,7924,122,1
125109,2835,1564,1
125110,2427,10124,1


In [12]:
#Reading BinaryInteraction InteractedOnly test data from pkl file
testDf = pd.read_pickle("../Data/pkl/1M/BinaryInteraction/InteractedOnly/Test.pkl")
testDf

,UserId,MovieId,Interaction
0,3331,70,1
1,9119,380,1
2,1411,530,1
3,8846,1582,1
4,4289,309,1
...,...,...,...
125107,5743,3202,1
125108,5950,1535,1
125109,213,4928,1
125110,4972,108,1


In [13]:
#Generating totalTestDf that include test and validation data
totalTestDf = pd.concat([validationDf, testDf], ignore_index = True)
totalTestDf

,UserId,MovieId,Interaction
0,718,1903,1
1,2830,3528,1
2,9138,179,1
3,4026,504,1
4,8566,7612,1
...,...,...,...
250219,5743,3202,1
250220,5950,1535,1
250221,213,4928,1
250222,4972,108,1


In [14]:
#Since represent users in training dataset more important than represent them in test dataset
#For each unique user appending training dataframe more row than totalTest dataframe
#Following loop just calculate which rows deleted from totalTestDf, adding rows a list and delete them from totalTestDf
#totalTestDf printing just for check its situation
rowList = []
removeIndex = []
for i in range(userSize):
    elementTrainingSize = trainingDf[trainingDf['UserId'] == i].shape[0]
    elementTestSize = totalTestDf[totalTestDf['UserId'] == i].shape[0]
        
    if elementTrainingSize < elementTestSize:
        transferItemSize = ((elementTrainingSize + elementTestSize) // 2) + 1 - elementTrainingSize
        transferItemIndex = totalTestDf[totalTestDf['UserId'] == i].index[:transferItemSize].tolist()
        removeIndex.extend(transferItemIndex)
        for j in transferItemIndex:
            rowList.append([totalTestDf.iloc[j].UserId, totalTestDf.iloc[j].MovieId, totalTestDf.iloc[j].Interaction])
totalTestDf.drop(removeIndex, inplace=True)
totalTestDf.reset_index(drop=True, inplace=True)
totalTestDf

,UserId,MovieId,Interaction
0,718,1903,1
1,2830,3528,1
2,9138,179,1
3,4026,504,1
4,8566,7612,1
...,...,...,...
249981,5743,3202,1
249982,5950,1535,1
249983,213,4928,1
249984,4972,108,1


In [15]:
len(rowList)

238

In [16]:
#Adding unrepresented users to training dataframe 
trainingDf = pd.concat([trainingDf, pd.DataFrame(rowList, columns=['UserId', 'MovieId', 'Interaction'])], ignore_index=True)
trainingDf

,UserId,MovieId,Interaction
0,9608,768,1
1,7750,2229,1
2,6702,3129,1
3,9108,208,1
4,9684,7155,1
...,...,...,...
750902,9839,2965,1
750903,9869,1260,1
750904,9889,427,1
750905,9924,146,1


In [17]:
#Checking is there at least 1 input for each user in training data
trainingDf['UserId'].unique().shape[0]

10073

In [18]:
#Since represent movies in training dataset more important than represent them in test dataset
#For each unique movie appending training dataframe more row than totalTest dataframe
#Following loop just calculate which rows deleted from totalTestDf, adding rows a list and delete them from totalTestDf
#totalTestDf printing just for check its situation
rowList = []
removeIndex = []
for i in range(movieSize):
    elementTrainingSize = trainingDf[trainingDf['MovieId'] == i].shape[0]
    elementTestSize = totalTestDf[totalTestDf['MovieId'] == i].shape[0]
    if elementTrainingSize < elementTestSize:
        transferItemSize = ((elementTrainingSize + elementTestSize) // 2) + 1 - elementTrainingSize
        transferItemIndex = totalTestDf[totalTestDf['MovieId'] == i].index[:transferItemSize].tolist()
        removeIndex.extend(transferItemIndex)
        for j in transferItemIndex:
            rowList.append([totalTestDf.iloc[j].UserId, totalTestDf.iloc[j].MovieId, totalTestDf.iloc[j].Interaction])
totalTestDf.drop(removeIndex, inplace=True)
totalTestDf.reset_index(drop=True, inplace=True)
totalTestDf

,UserId,MovieId,Interaction
0,718,1903,1
1,2830,3528,1
2,9138,179,1
3,4026,504,1
4,8566,7612,1
...,...,...,...
247343,5743,3202,1
247344,5950,1535,1
247345,213,4928,1
247346,4972,108,1


In [19]:
len(rowList)

2638

In [20]:
#Adding unrepresented movies to training dataframe 
trainingDf = pd.concat([trainingDf, pd.DataFrame(rowList, columns=['UserId', 'MovieId', 'Interaction'])], ignore_index=True)
trainingDf

,UserId,MovieId,Interaction
0,9608,768,1
1,7750,2229,1
2,6702,3129,1
3,9108,208,1
4,9684,7155,1
...,...,...,...
753540,9985,21998,1
753541,9998,22002,1
753542,9999,22008,1
753543,10037,22017,1


In [21]:
#Shuffle trainingDf
trainingDf = trainingDf.sample(frac=1).reset_index(drop=True)
trainingDf

,UserId,MovieId,Interaction
0,4356,1027,1
1,3946,236,1
2,1399,113,1
3,3078,1586,1
4,8244,806,1
...,...,...,...
753540,3299,795,1
753541,4805,3723,1
753542,8255,94,1
753543,5854,1077,1


In [22]:
#Checking is there at least 1 input for each movie in training data
trainingDf['MovieId'].unique().shape[0]

22033

In [23]:
#totalTest data is splitting as validation data and test data
validationDf, testDf = train_test_split(totalTestDf, test_size=0.5)
validationDf.reset_index(drop=True, inplace = True)
validationDf

,UserId,MovieId,Interaction
0,7693,399,1
1,9590,1096,1
2,8546,285,1
3,9622,588,1
4,3870,1342,1
...,...,...,...
123669,8317,517,1
123670,2133,25,1
123671,8927,694,1
123672,9450,88,1


In [24]:
#Resetting Index and Checking test data frame
testDf.reset_index(drop=True, inplace = True)
testDf

,UserId,MovieId,Interaction
0,6731,149,1
1,3630,626,1
2,205,387,1
3,6673,2122,1
4,9225,3848,1
...,...,...,...
123669,8043,446,1
123670,2928,3526,1
123671,7986,1796,1
123672,2024,2730,1


In [25]:
#Organized data save as pkl file for future uses
trainingDf.to_pickle("../Data/pkl/1M/Qualified/BinaryInteraction/InteractedOnly/Training.pkl")
validationDf.to_pickle("../Data/pkl/1M/Qualified/BinaryInteraction/InteractedOnly/Validation.pkl")
testDf.to_pickle("../Data/pkl/1M/Qualified/BinaryInteraction/InteractedOnly/Test.pkl")

In [26]:
#Reading Qualified BinaryInteraction InteractedOnly training data from pkl file
trainingDf = pd.read_pickle("../Data/pkl/1M/Qualified/BinaryInteraction/InteractedOnly/Training.pkl")
trainingDf

,UserId,MovieId,Interaction
0,4356,1027,1
1,3946,236,1
2,1399,113,1
3,3078,1586,1
4,8244,806,1
...,...,...,...
753540,3299,795,1
753541,4805,3723,1
753542,8255,94,1
753543,5854,1077,1


In [27]:
#Reading Qualified BinaryInteraction InteractedOnly validation data from pkl file
validationDf = pd.read_pickle("../Data/pkl/1M/Qualified/BinaryInteraction/InteractedOnly/Validation.pkl")
validationDf

,UserId,MovieId,Interaction
0,7693,399,1
1,9590,1096,1
2,8546,285,1
3,9622,588,1
4,3870,1342,1
...,...,...,...
123669,8317,517,1
123670,2133,25,1
123671,8927,694,1
123672,9450,88,1


In [28]:
#Reading Qualified BinaryInteraction InteractedOnly test data from pkl file
testDf = pd.read_pickle("../Data/pkl/1M/Qualified/BinaryInteraction/InteractedOnly/Test.pkl")
testDf

,UserId,MovieId,Interaction
0,6731,149,1
1,3630,626,1
2,205,387,1
3,6673,2122,1
4,9225,3848,1
...,...,...,...
123669,8043,446,1
123670,2928,3526,1
123671,7986,1796,1
123672,2024,2730,1


In [29]:
#Generating df that include training, test and validation data
df = pd.concat([trainingDf, validationDf, testDf], ignore_index = True)
df

,UserId,MovieId,Interaction
0,4356,1027,1
1,3946,236,1
2,1399,113,1
3,3078,1586,1
4,8244,806,1
...,...,...,...
1000888,8043,446,1
1000889,2928,3526,1
1000890,7986,1796,1
1000891,2024,2730,1


In [30]:
#Checking is there any nan value in df
df.isnull().values.any()

False

In [31]:
#Check if any duplicate user-movie pairs exist in df
df[['UserId', 'MovieId']].duplicated().any()

False

In [32]:
#Calculating number of users in Qualified Data
df['UserId'].unique().shape[0]

10073

In [33]:
#Calculating number of movies in Qualified Data
df['MovieId'].unique().shape[0]

22033

In [34]:
#Reading BinaryInteraction NotInteractedSample training data from pkl file
trainingDf = pd.read_pickle("../Data/pkl/1M/BinaryInteraction/NotInteractedSample/Training.pkl")
trainingDf

,UserId,MovieId,Interaction
0,234,3490,1
1,8276,2698,1
2,6124,18989,1
3,9040,4706,1
4,3843,1576,1
...,...,...,...
1000888,9095,2752,1
1000889,2024,4478,1
1000890,2464,15523,0
1000891,1578,1857,0


In [35]:
#Reading BinaryInteraction NotInteractedSample validation data from pkl file
validationDf = pd.read_pickle("../Data/pkl/1M/BinaryInteraction/NotInteractedSample/Validation.pkl")
validationDf

,UserId,MovieId,Interaction
0,4329,1456,1
1,1276,13547,0
2,4325,421,0
3,5368,5840,1
4,2501,80,1
...,...,...,...
166810,1023,17637,0
166811,7704,8313,1
166812,537,4776,0
166813,9358,385,1


In [36]:
#Reading BinaryInteraction NotInteractedSample test data from pkl file
testDf = pd.read_pickle("../Data/pkl/1M/BinaryInteraction/NotInteractedSample/Test.pkl")
testDf

,UserId,MovieId,Interaction
0,7677,2663,1
1,7202,741,1
2,2102,919,1
3,4105,1046,1
4,7207,165,1
...,...,...,...
166811,8917,62,1
166812,8582,227,1
166813,803,761,1
166814,4604,693,1


In [37]:
#Generating totalTestDf that include test and validation data
totalTestDf = pd.concat([validationDf, testDf], ignore_index = True)
totalTestDf

,UserId,MovieId,Interaction
0,4329,1456,1
1,1276,13547,0
2,4325,421,0
3,5368,5840,1
4,2501,80,1
...,...,...,...
333626,8917,62,1
333627,8582,227,1
333628,803,761,1
333629,4604,693,1


In [38]:
#Since represent users in training dataset more important than represent them in test dataset
#For each unique user appending training dataframe more row than totalTest dataframe
#Following loop just calculate which rows deleted from totalTestDf, adding rows a list and delete them from totalTestDf
#totalTestDf printing just for check its situation
rowList = []
removeIndex = []
for i in range(userSize):
    elementTrainingSize = trainingDf[trainingDf['UserId'] == i].shape[0]
    elementTestSize = totalTestDf[totalTestDf['UserId'] == i].shape[0]
        
    if elementTrainingSize < elementTestSize:
        transferItemSize = ((elementTrainingSize + elementTestSize) // 2) + 1 - elementTrainingSize
        transferItemIndex = totalTestDf[totalTestDf['UserId'] == i].index[:transferItemSize].tolist()
        removeIndex.extend(transferItemIndex)
        for j in transferItemIndex:
            rowList.append([totalTestDf.iloc[j].UserId, totalTestDf.iloc[j].MovieId, totalTestDf.iloc[j].Interaction])
totalTestDf.drop(removeIndex, inplace=True)
totalTestDf.reset_index(drop=True, inplace=True)
totalTestDf

,UserId,MovieId,Interaction
0,4329,1456,1
1,1276,13547,0
2,4325,421,0
3,5368,5840,1
4,2501,80,1
...,...,...,...
333624,8917,62,1
333625,8582,227,1
333626,803,761,1
333627,4604,693,1


In [39]:
len(rowList)

2

In [40]:
#Adding unrepresented users to training dataframe 
trainingDf = pd.concat([trainingDf, pd.DataFrame(rowList, columns=['UserId', 'MovieId', 'Interaction'])], ignore_index=True)
trainingDf

,UserId,MovieId,Interaction
0,234,3490,1
1,8276,2698,1
2,6124,18989,1
3,9040,4706,1
4,3843,1576,1
...,...,...,...
1000890,2464,15523,0
1000891,1578,1857,0
1000892,7899,219,1
1000893,9637,18227,0


In [41]:
#Checking is there at least 1 input for each user in training data
trainingDf['UserId'].unique().shape[0]

10073

In [42]:
#Since represent movies in training dataset more important than represent them in test dataset
#For each unique movie appending training dataframe more row than totalTest dataframe
#Following loop just calculate which rows deleted from totalTestDf, adding rows a list and delete them from totalTestDf
#totalTestDf printing just for check its situation
rowList = []
removeIndex = []
for i in range(movieSize):
    elementTrainingSize = trainingDf[trainingDf['MovieId'] == i].shape[0]
    elementTestSize = totalTestDf[totalTestDf['MovieId'] == i].shape[0]
    if elementTrainingSize < elementTestSize:
        transferItemSize = ((elementTrainingSize + elementTestSize) // 2) + 1 - elementTrainingSize
        transferItemIndex = totalTestDf[totalTestDf['MovieId'] == i].index[:transferItemSize].tolist()
        removeIndex.extend(transferItemIndex)
        for j in transferItemIndex:
            rowList.append([totalTestDf.iloc[j].UserId, totalTestDf.iloc[j].MovieId, totalTestDf.iloc[j].Interaction])
totalTestDf.drop(removeIndex, inplace=True)
totalTestDf.reset_index(drop=True, inplace=True)
totalTestDf

,UserId,MovieId,Interaction
0,4329,1456,1
1,1276,13547,0
2,4325,421,0
3,5368,5840,1
4,2501,80,1
...,...,...,...
333383,8917,62,1
333384,8582,227,1
333385,803,761,1
333386,4604,693,1


In [43]:
len(rowList)

241

In [44]:
#Adding unrepresented movies to training dataframe 
trainingDf = pd.concat([trainingDf, pd.DataFrame(rowList, columns=['UserId', 'MovieId', 'Interaction'])], ignore_index=True)
trainingDf

,UserId,MovieId,Interaction
0,234,3490,1
1,8276,2698,1
2,6124,18989,1
3,9040,4706,1
4,3843,1576,1
...,...,...,...
1001131,5887,21774,0
1001132,4768,21856,0
1001133,7387,21856,0
1001134,9696,21952,0


In [45]:
#Shuffle trainingDf
trainingDf = trainingDf.sample(frac=1).reset_index(drop=True)
trainingDf

,UserId,MovieId,Interaction
0,5282,244,1
1,766,213,1
2,8209,1198,1
3,1067,1828,1
4,6915,1256,1
...,...,...,...
1001131,10072,1095,1
1001132,133,396,1
1001133,8852,6102,1
1001134,5174,4218,0


In [46]:
#Checking is there at least 1 input for each movie in training data
trainingDf['MovieId'].unique().shape[0]

22033

In [47]:
#totalTest data is splitting as validation data and test data
validationDf, testDf = train_test_split(totalTestDf, test_size=0.5)
validationDf.reset_index(drop=True, inplace = True)
validationDf

,UserId,MovieId,Interaction
0,8455,142,1
1,1945,530,1
2,7588,4030,1
3,6186,3563,1
4,1088,8835,1
...,...,...,...
166689,8048,1278,1
166690,5234,4508,0
166691,9471,19733,0
166692,8078,15906,0


In [48]:
#Resetting Index and Checking test data frame
testDf.reset_index(drop=True, inplace = True)
testDf

,UserId,MovieId,Interaction
0,604,444,1
1,4908,10378,1
2,6086,865,1
3,1424,4067,0
4,6893,1406,1
...,...,...,...
166689,5750,1053,1
166690,5146,18003,0
166691,9484,9793,1
166692,2433,491,1


In [49]:
#Organized data save as pkl file for future uses
trainingDf.to_pickle("../Data/pkl/1M/Qualified/BinaryInteraction/NotInteractedSample/Training.pkl")
validationDf.to_pickle("../Data/pkl/1M/Qualified/BinaryInteraction/NotInteractedSample/Validation.pkl")
testDf.to_pickle("../Data/pkl/1M/Qualified/BinaryInteraction/NotInteractedSample/Test.pkl")

In [50]:
#Reading Qualified BinaryInteraction NotInteractedSample training data from pkl file
trainingDf = pd.read_pickle("../Data/pkl/1M/Qualified/BinaryInteraction/NotInteractedSample/Training.pkl")
trainingDf

,UserId,MovieId,Interaction
0,5282,244,1
1,766,213,1
2,8209,1198,1
3,1067,1828,1
4,6915,1256,1
...,...,...,...
1001131,10072,1095,1
1001132,133,396,1
1001133,8852,6102,1
1001134,5174,4218,0


In [51]:
#Reading Qualified BinaryInteraction NotInteractedSample validation data from pkl file
validationDf = pd.read_pickle("../Data/pkl/1M/Qualified/BinaryInteraction/NotInteractedSample/Validation.pkl")
validationDf

,UserId,MovieId,Interaction
0,8455,142,1
1,1945,530,1
2,7588,4030,1
3,6186,3563,1
4,1088,8835,1
...,...,...,...
166689,8048,1278,1
166690,5234,4508,0
166691,9471,19733,0
166692,8078,15906,0


In [52]:
#Reading Qualified BinaryInteraction NotInteractedSample test data from pkl file
testDf = pd.read_pickle("../Data/pkl/1M/Qualified/BinaryInteraction/NotInteractedSample/Test.pkl")
testDf

,UserId,MovieId,Interaction
0,604,444,1
1,4908,10378,1
2,6086,865,1
3,1424,4067,0
4,6893,1406,1
...,...,...,...
166689,5750,1053,1
166690,5146,18003,0
166691,9484,9793,1
166692,2433,491,1


In [53]:
#Generating df that include training, test and validation data
df = pd.concat([trainingDf, validationDf, testDf], ignore_index = True)
df

,UserId,MovieId,Interaction
0,5282,244,1
1,766,213,1
2,8209,1198,1
3,1067,1828,1
4,6915,1256,1
...,...,...,...
1334519,5750,1053,1
1334520,5146,18003,0
1334521,9484,9793,1
1334522,2433,491,1


In [54]:
#Checking is there any nan value in df
df.isnull().values.any()

False

In [55]:
#Check if any duplicate user-movie pairs exist in df
df[['UserId', 'MovieId']].duplicated().any()

False

In [56]:
#Calculating number of users in Qualified Data
df['UserId'].unique().shape[0]

10073

In [57]:
#Calculating number of movies in Qualified Data
df['MovieId'].unique().shape[0]

22033